In [36]:
### Supposedly: Run this block first, then sbatch the exported .slurm file through the console
### But it doesn't actually work due to errors activating conda through the cluster(?)
### For now, skip this block

import multiprocessing

print(multiprocessing.cpu_count()) #num of available cpus

import os
os.system('jupyter nbconvert --to python sklearn-parallel.ipynb') # Converts the nb into .py

job_name = "rfr_job"
sbatch_file = f"{job_name}.slurm"

### Creating a sbatch file to run the .py 
with open(sbatch_file, "w") as f:
    f.write("#!/bin/bash\n")
    f.write(f"#SBATCH --job-name={job_name}\n")
    f.write("#SBATCH --output=output.out\n")
    f.write("#SBATCH --account=csd709\n") # Change account number to new one
    f.write("#SBATCH --nodes=1\n")
    f.write("#SBATCH --ntasks-per-node=10\n")
    f.write("#SBATCH --cpus-per-task=5\n")
    f.write("#SBATCH --time=01:30:00\n")
    f.write("#SBATCH --mem=32G\n")
    f.write("#SBATCH --partition=shared\n")
    f.write("\n")
    f.write("module purge\n")
    f.write("module load slurm\n")
    f.write("module load cpu/0.15.4  gcc/10.2.0\n")
    #f.write("module load gpu/0.15.4\n") 
    f.write("module load python/3.8.5\n")
    f.write("module load singularitypro/3.7\n")
    f.write("module load anaconda3/2020.11/conda\n")
    f.write("echo $CONDA_PYTHON_EXE\n")
    f.write("conda activate base\n")
    f.write("\n")    
    f.write("python ./sklearn-parallel.py\n")
# Python outputs should be found in the *.out file  

os.system(f"sbatch {sbatch_file}")

128


32512

In [1]:
#### Data preperation
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


#Load and preprocess the data
folder_path = '100 files'
data = {}

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            df1 = pd.read_csv(file_path, header=None, names=['des_flux', 'detrap_en', 'def_conc'], index_col=None, squeeze = True)
            #df1 = df1.fillna(df1.mean()) # Replace NaN with column average
            data[filename] = df1
            
# Combine all the data into a single dataframe
df = pd.concat(data.values())

des_flux=df['des_flux']
des_flux=des_flux.tolist()
num_files=int(len(des_flux)/301)
des_flux=np.reshape(des_flux, [num_files, 301])
display(des_flux)
#print(type(des_flux[0]), des_flux.shape)

detrap_en=df['detrap_en']
detrap_en=detrap_en.dropna()
detrap_en=detrap_en.tolist()
num_files=int(len(detrap_en)/4)
detrap_en=np.reshape(detrap_en, [num_files, 4])
#print(detrap_en, detrap_en.shape)

def_conc=df['def_conc']
def_conc=def_conc.dropna()
def_conc=def_conc.tolist()
num_files=int(len(def_conc)/4)
def_conc=np.reshape(def_conc, [num_files, 4])
#print(def_conc, def_conc.shape)

def_param = np.concatenate([detrap_en, def_conc], axis=1)
print(def_param.shape)

# Split the data into train and test sets
X_train, X_test= train_test_split(des_flux, test_size=0.1, random_state=42)
y_train, y_test = train_test_split(def_param, test_size=0.1, random_state=42)
display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Normalize the data
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_norm = scaler.fit_transform(X_train.reshape(-1, 1))
X_test_norm = scaler.transform(X_test.reshape(-1, 1))

#Reshape data for model input
#Checked to make sure the time-series nature of the X_train, X_test is intact, can double check
#By calling an array and comparing with a .csv file
X_train_transform = np.reshape(X_train_norm, (X_train.shape))
print(X_train_transform.shape)

X_test_transform = np.reshape(X_test_norm, (X_test.shape))


array([[ 0.00000000e+00,  9.12392689e+05,  1.68597525e+06, ...,
         2.78461900e+12,  1.00358797e+12,  2.97558333e+11],
       [-5.47779602e-12,  5.35972333e+13,  8.17158826e+13, ...,
         1.16331140e+09,  8.23480169e+08,  4.26582636e+08],
       [ 0.00000000e+00,  1.68268296e+13,  2.62041737e+13, ...,
         3.31027987e+10,  3.98052117e+10,  2.57056280e+10],
       ...,
       [ 0.00000000e+00,  4.23774406e+15,  5.88839228e+15, ...,
        -8.42365120e+10, -5.29676771e+10, -3.64623675e+10],
       [ 0.00000000e+00,  3.96142323e+12,  6.22212473e+12, ...,
         1.09171263e+12, -1.63494357e+12, -3.20546057e+11],
       [ 4.32462462e-18,  4.07106353e+08,  7.05768695e+08, ...,
         7.50076315e+10,  1.21111702e+11,  2.06857976e+11]])

(100, 8)


(90, 301)

(10, 301)

(90, 8)

(10, 8)

(90, 301)


In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import plot_tree
from joblib import Parallel, delayed
from joblib import parallel_backend

### Defining a function to train a random forest with gridsearchcv
def train_rf(X, y, i):
    rfr = RandomForestRegressor(random_state=42, max_depth=10)
    param_grid= {"n_estimators": range(1,50), "max_depth": range(1,20)}
    folds = KFold(n_splits=10, shuffle=True, random_state=42)

    
    with parallel_backend('threading'): # threading for inner loop
        rf_gcv = GridSearchCV(rfr, param_grid, 
                              return_train_score=True,
                              scoring="neg_mean_squared_error",
                              cv=folds, n_jobs=50)
    
        rf_gcv.fit(X, y)

    print(rf_gcv.best_params_)
    print(rf_gcv.best_score_)
    
    # Predict based on test data 
    best_gcv = rf_gcv.best_estimator_
    rfr_y_predict = best_gcv.predict(X)

    # Calculate the mse
    mse = mean_squared_error(y, rfr_y_predict)

    # Print the MSE for this model
    print(f"Model {i} MSE: {mse}")

    return rf_gcv


In [ ]:
%%time
## Method 1
# Define the number of models to train in parallel
num_models = 10

# Parallelize the model training using sklearn joblib
models = Parallel(n_jobs=num_models)(
    delayed(train_rf)(X_train_transform, y_train, i) for i in range(num_models))

# Combine the predictions from all the models
y_pred = np.mean([model.predict(X_test_transform) for model in models], axis=0)

# Calculate the final mean squared error
mse = mean_squared_error(y_test, y_pred)

# Print the final MSE
print(f"Final MSE = {mse}")


In [16]:
%%time
## Method 2
num_models = 10
with parallel_backend('multiprocessing', n_jobs=-1): # multiprocessing for outloop, prevents the parallel nested loop error
    models = Parallel()(delayed(train_rf)(X_train_transform, y_train, i) for i in range(num_models))

# Combines predictions from all the models by taking the mean
y_pred = np.mean([model.predict(X_test_transform) for model in models], axis=0)

# Calculates final MSE
mse = mean_squared_error(y_test, y_pred)

# Prints final MSE
print(f"Final MSE = {mse}")

{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 9 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 2 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 4 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 0 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 6 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 1 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 8 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 3 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 5 MSE: 0.0010520446377357594
{'max_depth': 7, 'n_estimators': 49}
-0.006440878423247848
Model 7 MSE: 0.0010520446377357594
Final MSE = 0.006309377060303989
CPU times: user 157 ms, sys

In [5]:
### Basic PLSR
from sklearn.cross_decomposition import PLSRegression
plsr = PLSRegression(n_components=2)
plsr.fit(X_train_transform, y_train)
X_train_pls = plsr.transform(X_train_transform)
pls_pred = plsr.predict(X_test_transform)
# Calculates mse 
mse = mean_squared_error(y_test, pls_pred)
print(f"MSE = {mse}")

MSE = 0.007196555368356303
